In [ ]:
# @title 1. Install Dependencies
# Installs vectorbt for fast backtesting/calculation and yfinance for data
%pip install vectorbt yfinance streamlit plotly -q

import vectorbt as vbt
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime

# Configuration
SYMBOL = "IDEA.NS"      # Target Stock
HISTORY_PERIOD = "1y"   # Sufficient for indicators
HISTORY_INTERVAL = "1d" # Base trend
LIVE_PERIOD = "1d"      # Fetch last day for intraday update
LIVE_INTERVAL = "1m"    # Granular live data

warnings.filterwarnings('ignore')
print("✅ Libraries Installed & Configured.")

Note: you may need to restart the kernel to use updated packages.
✅ Libraries Installed & Configured.


In [2]:
# @title 2. Data Engine Functions
def get_hybrid_data(symbol):
    print(f"\n🚀 FETCHING HYBRID DATA FOR {symbol}...")

    # 1. Download History
    df_hist = yf.download(symbol, period=HISTORY_PERIOD, interval=HISTORY_INTERVAL, progress=False)

    # Clean MultiIndex (Common yfinance issue)
    if isinstance(df_hist.columns, pd.MultiIndex):
        df_hist.columns = df_hist.columns.droplevel(1)

    # Ensure timezone naive for easy merging
    if df_hist.index.tz is not None:
        df_hist.index = df_hist.index.tz_localize(None)

    # 2. Download Live Minute Data (Snapshot)
    try:
        df_live = yf.download(symbol, period=LIVE_PERIOD, interval=LIVE_INTERVAL, progress=False)
        if isinstance(df_live.columns, pd.MultiIndex):
            df_live.columns = df_live.columns.droplevel(1)

        if not df_live.empty:
            # Construct the "Current Daily Candle" from minute data
            latest_price = df_live['Close'].iloc[-1]
            live_candle = pd.DataFrame({
                'Open': [df_live['Open'].iloc[0]],
                'High': [df_live['High'].max()],
                'Low': [df_live['Low'].min()],
                'Close': [latest_price],
                'Volume': [df_live['Volume'].sum()]
            }, index=[pd.Timestamp.now(tz='Asia/Kolkata').normalize().tz_localize(None)]) # Normalize to midnight to match daily index

            # Remove today from history if it exists (to replace with live version)
            today_date = pd.Timestamp.now(tz='Asia/Kolkata').normalize().tz_localize(None)
            if today_date in df_hist.index:
                df_hist = df_hist.drop(today_date)

            # Merge
            df_final = pd.concat([df_hist, live_candle])
            print(f"✅ Live Data Merged. Current Price: {latest_price:.2f}")
        else:
            df_final = df_hist
            print("⚠️ Market closed or no live data. Using closing history.")

    except Exception as e:
        print(f"⚠️ Live Fetch Error: {e}")
        df_final = df_hist

    df_final = df_final.ffill() # Ensure no gaps

    # --- NEW ADDITION: PRINT OHLCV + INDICATORS ---
    try:
        # Calculate temporary indicators just for the snapshot display
        close = df_final['Close']
        rsi_snap = vbt.RSI.run(close).rsi
        sma_50_snap = vbt.MA.run(close, 50).ma
        sma_200_snap = vbt.MA.run(close, 200).ma

        print(f"\n📊 LATEST MARKET SNAPSHOT ({pd.Timestamp.now().date()}):")
        print("-" * 100) # Made line longer to fit more columns

        # Create a DataFrame with OHLCV AND Indicators
        snapshot_df = pd.DataFrame({
            'Open': df_final['Open'],
            'High': df_final['High'],
            'Low': df_final['Low'],
            'Close': df_final['Close'],
            'Volume': df_final['Volume'],
            'RSI': rsi_snap,
            'SMA_50': sma_50_snap,
            'SMA_200': sma_200_snap
        }).tail(3) # Get the last 3 rows

        print(snapshot_df)
        print("-" * 100)
    except Exception as e:
        print(f"⚠️ Could not print indicator snapshot: {e}")
    # ----------------------------------------------

    return df_final

In [3]:
# @title 3. VectorBT Logic & Signal Generation
def run_vector_strategy(df):
    close = df['Close']

    # --- 1. INDICATOR CALCULATION ---
    # RSI
    rsi = vbt.RSI.run(close, window=14)
    df['RSI'] = rsi.rsi

    # MACD
    macd = vbt.MACD.run(close, fast_window=12, slow_window=26, signal_window=9)
    df['MACD'] = macd.macd
    df['MACD_Signal'] = macd.signal
    df['MACD_Hist'] = macd.hist

    # Bollinger Bands
    bbands = vbt.BBANDS.run(close, window=20)
    df['BB_Upper'] = bbands.upper
    df['BB_Lower'] = bbands.lower
    df['BB_Mid'] = bbands.middle
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Mid']

    # SMAs (Context)
    df['SMA_50'] = vbt.MA.run(close, window=50).ma
    df['SMA_200'] = vbt.MA.run(close, window=200).ma

    # --- 2. VECTORIZED SIGNAL LOGIC (UPDATED) ---

    # Standard Conditions
    cond_rsi_buy = df['RSI'] < 30
    cond_rsi_sell = df['RSI'] > 70
    cond_bb_dip = close < df['BB_Lower']

    # --- NEW: MACD MOMENTUM FILTER ---
    # We check if the Histogram is higher than it was yesterday.
    # This means the bearish momentum is slowing down.
    cond_macd_improving = df['MACD_Hist'] > df['MACD_Hist'].shift(1)

    # Initialize Signal Column
    df['Signal_Score'] = 0

    # LOGIC A: STRONG BUY (The "Perfect Setup")
    # Oversold + Below Bands + Momentum Turning Up
    df.loc[cond_rsi_buy & cond_bb_dip & cond_macd_improving, 'Signal_Score'] = 2

    # LOGIC B: BUY (The "Standard Setup")
    # Oversold + Momentum Turning Up (but not below bands)
    df.loc[cond_rsi_buy & ~cond_bb_dip & cond_macd_improving, 'Signal_Score'] = 1

    # LOGIC C: SELL
    # Overbought (We keep this simple for now, usually you sell on weakness)
    df.loc[cond_rsi_sell, 'Signal_Score'] = -1

    return df
def generate_report(df):
    last = df.iloc[-1]
    score = last['Signal_Score']

    # Map score to text
    if score == 2: signal_txt = "STRONG BUY 🚀"
    elif score == 1: signal_txt = "BUY 🟢"
    elif score == -1: signal_txt = "SELL 🔴"
    else: signal_txt = "NEUTRAL ⚪"

    print(f"\n📊 DASHBOARD SNAPSHOT [{df.index[-1].date()}]")
    print("="*40)
    print(f"Price:      {last['Close']:.2f}")
    print(f"RSI:        {last['RSI']:.2f}")
    print(f"BB Width:   {last['BB_Width']:.4f}")
    print("="*40)
    print(f"SIGNAL:     {signal_txt}")
    print("="*40)
    return signal_txt

In [4]:
# @title 4. Visualization Engine
def plot_dashboard(df, symbol, signal_txt):
    # Slice to last 150 candles for clarity
    subset = df.tail(150)

    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.6, 0.2, 0.2],
        subplot_titles=(f"{symbol} Price Action", "RSI Momentum", "MACD Trend")
    )

    # --- ROW 1: Price & BB ---
    # BB Cloud
    fig.add_trace(go.Scatter(x=subset.index, y=subset['BB_Upper'], line=dict(width=1, color='gray'), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=subset.index, y=subset['BB_Lower'], line=dict(width=1, color='gray'), fill='tonexty', fillcolor='rgba(128,128,128,0.1)', showlegend=False), row=1, col=1)

    # Candles
    fig.add_trace(go.Candlestick(x=subset.index, open=subset['Open'], high=subset['High'], low=subset['Low'], close=subset['Close'], name='Price'), row=1, col=1)

    # SMAs
    fig.add_trace(go.Scatter(x=subset.index, y=subset['SMA_50'], line=dict(color='orange', width=1), name='SMA 50'), row=1, col=1)
    fig.add_trace(go.Scatter(x=subset.index, y=subset['SMA_200'], line=dict(color='blue', width=1), name='SMA 200'), row=1, col=1)

    # --- ROW 2: RSI ---
    fig.add_trace(go.Scatter(x=subset.index, y=subset['RSI'], line=dict(color='#A100FF', width=2), name='RSI'), row=2, col=1)
    fig.add_hline(y=70, line_dash="dot", row=2, col=1, line_color="red")
    fig.add_hline(y=30, line_dash="dot", row=2, col=1, line_color="green")

    # --- ROW 3: MACD ---
    colors = np.where(subset['MACD_Hist'] < 0, 'red', 'green')
    fig.add_trace(go.Bar(x=subset.index, y=subset['MACD_Hist'], marker_color=colors, name='MACD Hist'), row=3, col=1)
    fig.add_trace(go.Scatter(x=subset.index, y=subset['MACD'], line=dict(color='cyan', width=1), name='MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=subset.index, y=subset['MACD_Signal'], line=dict(color='orange', width=1), name='Signal'), row=3, col=1)

    # Layout Optimization
    fig.update_layout(
        template="plotly_dark",
        height=900,
        title_text=f"Live Analysis: {symbol} | Signal: {signal_txt}",
        xaxis_rangeslider_visible=False,
        hovermode="x unified" # <--- Key optimization for readability
    )

    # Fix Y-axis scaling issues with BBands
    fig.update_yaxes(autorange=True, fixedrange=False)
    # Filter only Buy signals for plotting
    buys = subset[subset['Signal_Score'] > 0]
    # Add Green Triangles where we Buy
    fig.add_trace(go.Scatter(
        x=buys.index,
        y=buys['Low'] * 0.98, # Plot slightly below the candle
        mode='markers',
        marker=dict(symbol='triangle-up', color='#00FF00', size=12),
        name='Buy Signal'), row=1, col=1)

    fig.show()

In [5]:
# @title 5. Run Analysis
# 1. Get Data
data = get_hybrid_data(SYMBOL)

# 2. Run Strategy (Vectorized)
data = run_vector_strategy(data)

# 3. Generate Text Report
sig_text = generate_report(data)

# 4. Plot
plot_dashboard(data, SYMBOL, sig_text)


🚀 FETCHING HYBRID DATA FOR IDEA.NS...
✅ Live Data Merged. Current Price: 11.54

📊 LATEST MARKET SNAPSHOT (2026-01-06):
----------------------------------------------------------------------------------------------------
             Open   High    Low  Close      Volume        RSI   SMA_50  \
2026-01-02  11.72  12.05  11.64  11.78   372681156  55.584405  10.5878   
2026-01-05  11.80  12.02  11.36  11.43  1008782246  52.195125  10.6170   
2026-01-06  11.45  11.65  11.32  11.54   490208129  55.012223  10.6590   

            SMA_200  
2026-01-02  8.18290  
2026-01-05  8.20535  
2026-01-06  8.22585  
----------------------------------------------------------------------------------------------------

📊 DASHBOARD SNAPSHOT [2026-01-06]
Price:      11.54
RSI:        55.01
BB Width:   0.1409
SIGNAL:     NEUTRAL ⚪


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [6]:
# @title 6. Backtesting Engine (Did we make money?)
def run_backtest(df):
    print("\n💰 RUNNING PORTFOLIO SIMULATION...")

    # 1. Define Entry & Exit Logic
    # Entry: Any Buy Signal (Score 1 or 2)
    entries = df['Signal_Score'] > 0

    # Exit: Any Sell Signal (Score -1)
    exits = df['Signal_Score'] < 0

    # 2. Build the Portfolio
    # init_cash: Starting capital (e.g., ₹100,000)
    # fees: 0.001 = 0.1% (Approx brokerage + STT)
    # freq: '1D' (Daily data)
    pf = vbt.Portfolio.from_signals(
        df['Close'],
        entries,
        exits,
        init_cash=100000,
        fees=0.001,
        freq='1D'
    )

    # 3. Print Key Metrics
    print(f"Total Return:       {pf.total_return() * 100:.2f}%")
    print(f"Win Rate:           {pf.stats()['Win Rate [%]']:.2f}%")
    print(f"Max Drawdown:       {pf.max_drawdown() * 100:.2f}%")
    print(f"Sharpe Ratio:       {pf.sharpe_ratio():.2f}")
    print("-" * 30)

    # 4. Plot Equity Curve
    # This shows your account balance growing (or shrinking) over time
    pf.plot().show()

    return pf

# Execute
pf = run_backtest(data)


💰 RUNNING PORTFOLIO SIMULATION...
Total Return:       10.59%
Win Rate:           50.00%
Max Drawdown:       -22.99%
Sharpe Ratio:       0.54
------------------------------


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# @title 7. Parameter Optimization (Grid Search) - FIXED
def optimize_rsi(df):
    print("\n🔬 RUNNING PARAMETER OPTIMIZATION...")

    # 1. Define the Parameters to Test
    buy_thresholds = np.arange(10, 50, 5)

    # 2. Prepare Data for Broadcasting
    rsi_values = df['RSI'].values.reshape(-1, 1)
    threshold_values = buy_thresholds.reshape(1, -1)

    # 3. Generate Signals Matrix
    entries = rsi_values < threshold_values
    exits = rsi_values > 70

    # 4. Run Batch Simulation
    pf = vbt.Portfolio.from_signals(
        df['Close'],
        entries,
        exits,
        init_cash=100000,
        fees=0.001,
        freq='1D'
    )

    # 5. Analyze Results
    returns = pf.total_return() * 100

    # --- FIX: Use Numpy for Position Finding ---
    # We use np.argmax to get the exact integer position (0, 1, 2...)
    # This avoids confusion between Index Labels and Array Positions
    winner_idx = np.argmax(returns.values)
    best_threshold = buy_thresholds[winner_idx]
    best_return = returns.values[winner_idx]
    # -------------------------------------------

    print(f"🏆 BEST PARAMETER FOUND: RSI < {best_threshold}")
    print(f"📈 Potential Return: {best_return:.2f}%")
    print("-" * 30)

    # Leaderboard
    results_df = pd.DataFrame({
        'Threshold': buy_thresholds,
        'Return %': returns.values
    })
    print("Top 5 Settings:")
    print(results_df.sort_values(by='Return %', ascending=False).head(5))

    # 6. Visualize the Curve
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=buy_thresholds,
        y=returns.values,
        mode='lines+markers',
        marker=dict(size=10, color='gold'),
        line=dict(color='#A100FF')
    ))
    fig.update_layout(
        title="Optimization Curve: RSI Buy Threshold vs. Profit",
        xaxis_title="RSI Threshold (Lower is stricter)",
        yaxis_title="Total Return (%)",
        template="plotly_dark"
    )
    fig.show()

# Execute
optimize_rsi(data)

In [5]:
# @title 8. Alert Engine (Email & Pop-ups) - FINAL ROBUST VERSION
import smtplib
import time
from email.mime.text import MIMEText
from datetime import datetime
import sys

# --- CONFIGURATION (ENTER DETAILS SAFELY) ---
#SYMBOL = "IDEA.NS"
# 1. Generate a NEW App Password from Google
# 2. Paste it inside the quotes below
EMAIL_SENDER = "saikarthikreddykuppireddy@gmail.com"
EMAIL_PASSWORD = "mmci cuhh kwbn sdgo" 
EMAIL_RECEIVER = "saikarthikkuppireddy@gmail.com"

# --- 1. EMAIL ALERT FUNCTION ---
def send_email_alert(symbol, signal_type, price):
    print(f"📧 Sending Email Alert for {signal_type}...")
    try:
        subject = f"🚨 TRADING ALERT: {signal_type} {symbol}"
        body = f"""
        ----------------------------------------
        URGENT TRADING ALERT
        ----------------------------------------
        SIGNAL:   {signal_type}
        SYMBOL:   {symbol}
        PRICE:    {price:.2f}
        TIME:     {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        ----------------------------------------
        The strategy has detected a valid entry/exit.
        Check your broker immediately.
        """
        
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECEIVER

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        
        print(f"✅ Email Successfully Sent to {EMAIL_RECEIVER}")
        return True
        
    except Exception as e:
        print(f"❌ EMAIL FAILED. Error: {e}")
        return False

# --- 2. POP-UP ALERT FUNCTION (SMART) ---
def show_popup_alert(symbol, signal_type, price):
    """
    Attempts to show a window pop-up. 
    If running on Google Colab (where windows are impossible), 
    it prints a big visible ASCII banner instead.
    """
    msg_title = f"🚨 MARKET ALERT: {symbol}"
    msg_body = f"SIGNAL: {signal_type}\nPRICE: {price:.2f}"
    
    # Method A: Try Standard Pop-up (Works on Local PC)
    try:
        import tkinter as tk
        from tkinter import messagebox
        
        # Initialize hidden root window
        root = tk.Tk()
        root.withdraw()
        root.attributes("-topmost", True)
        
        messagebox.showwarning(title=msg_title, message=msg_body)
        root.destroy()
        print("✅ Desktop Pop-up Displayed")
        
    # Method B: Fallback for Google Colab / Headless Servers
    except Exception:
        print("\n" + "!"*40)
        print(f" {msg_title} ")
        print(f" {msg_body} ")
        print("!"*40 + "\n")
        print("ℹ️ (Desktop pop-ups disabled on Colab)")

# --- 3. THE LIVE MONITOR LOOP ---
def start_live_monitor(symbol, interval_seconds=60):
    print(f"\n👀 WATCHTOWER ACTIVE: Monitoring {symbol} every {interval_seconds}s...")
    print("Press Ctrl+C to stop.")
    
    # Initialize "Memory" to 0 (Neutral)
    last_signal = 0 
    
    try:
        while True:
            # -----------------------------------------
            # A. FETCH DATA & RUN STRATEGY
            # -----------------------------------------
            # We call the functions from Snippet 2 and 3
            try:
                data = get_hybrid_data(symbol)      
                data = run_vector_strategy(data)    
            except Exception as e:
                print(f"⚠️ Data Fetch Error: {e}. Retrying in 10s...")
                time.sleep(10)
                continue

            # -----------------------------------------
            # B. ANALYZE LATEST CANDLE
            # -----------------------------------------
            current_row = data.iloc[-1]
            current_signal = int(current_row['Signal_Score']) # Convert to integer
            current_price = float(current_row['Close'])       # Convert to float
            
            # Print Heartbeat
            t_str = datetime.now().strftime("%H:%M:%S")
            print(f"[{t_str}] Price: {current_price:.2f} | Score: {current_signal}")

            # -----------------------------------------
            # C. DECISION LOGIC (CHANGE DETECTION)
            # -----------------------------------------
            
            # TRIGGER 1: Buy Signal (Score 2) - Only if we weren't already buying
            if current_signal == 2 and last_signal != 2:
                print("\n🚀 STRONG BUY DETECTED! INITIATING ALERTS...")
                show_popup_alert(symbol, "STRONG BUY", current_price)
                send_email_alert(symbol, "STRONG BUY", current_price)
                
            # TRIGGER 2: Sell Signal (Score -1) - Only if we weren't already selling
            elif current_signal == -1 and last_signal != -1:
                print("\n🔴 SELL SIGNAL DETECTED! INITIATING ALERTS...")
                show_popup_alert(symbol, "SELL", current_price)
                send_email_alert(symbol, "SELL", current_price)
                
            # TRIGGER 3: Buy Signal (Score 1) - Only if we weren't already buying
            elif current_signal == 1 and last_signal != 1:
                print("\n🔴 BUY SIGNAL DETECTED! INITIATING ALERTS...")
                show_popup_alert(symbol, "BUY", current_price)
                send_email_alert(symbol, "BUY", current_price)
                
            # TRIGGER 4: Neutral Signal (Score 0) - stable with neither buy nor sell
            elif current_signal == 0 and last_signal != 0:
                print("\n🔴 NO SIGNAL DETECTED! INITIATING ALERTS...")
                show_popup_alert(symbol, "NEUTRAL", current_price)
                send_email_alert(symbol, "NEUTRAL", current_price)
            # Update Memory
            last_signal = current_signal
            
            # Wait for next cycle
            time.sleep(interval_seconds)
            
    except KeyboardInterrupt:
        print("\n🛑 Monitor Stopped by User.")

# --- 4. EXECUTION ---
# Uncomment ONE of the following lines:

# OPTION A: Test the alerts immediately (Run this once to verify email)
#send_email_alert("SYMBOL", "BUY", 100)
show_popup_alert("SYMBOL", "BUY", 100)

# OPTION B: Start the Real Monitor
#start_live_monitor("IDEA.NS", interval_seconds=60)

✅ Desktop Pop-up Displayed


In [ ]:
# --------dashboard part------------

%%writefile app.py
import streamlit as st
import time
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText
from datetime import datetime

# ==========================================
# 1. CONFIGURATION (ENTER DETAILS HERE)
# ==========================================
EMAIL_SENDER = "saikarthikreddykuppireddy@gmail.com"
EMAIL_PASSWORD = "mmci cuhh kwbn sdgo"  # <--- UPDATE THIS
EMAIL_RECEIVER = "saikarthikkuppireddy@gmail.com"

# ==========================================
# 2. EMAIL ALERT ENGINE
# ==========================================
def send_email_alert(symbol, signal_type, price):
    """Sends an email when a trade signal occurs."""
    try:
        subject = f"🚨 TRADING ALERT: {signal_type} {symbol}"
        body = f"""
        URGENT TRADING ALERT
        --------------------
        SIGNAL: {signal_type}
        SYMBOL: {symbol}
        PRICE:  {price:.2f}
        TIME:   {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        --------------------
        Check your broker immediately.
        """
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECEIVER

        # Connect to Gmail
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())

        return True # Success
    except Exception as e:
        return f"Error: {e}" # Return error message

# ==========================================
# 3. DASHBOARD UI SETUP
# ==========================================
st.set_page_config(page_title="Trading Bot Dashboard", layout="wide")

st.title("🚀 Algo-Trading Watchtower")

# --- SIDEBAR CONTROLS ---
st.sidebar.header("⚙️ Alert Configuration")
enable_alerts = st.sidebar.toggle("Enable Master Alerts", value=True)

st.sidebar.subheader("Trigger Settings")
check_buy = st.sidebar.checkbox("Alert on BUY Signal", value=True)
check_sell = st.sidebar.checkbox("Alert on SELL Signal", value=True)
check_rsi = st.sidebar.checkbox("Alert on RSI Extremes", value=False)

st.sidebar.markdown("---")
st.sidebar.info(f"📧 Alerts sent to:\n{EMAIL_RECEIVER}")

# --- MAIN SCREEN STATUS ---
col1, col2, col3 = st.columns(3)
with col1:
    st.metric(label="Market Status", value="OPEN (Simulated)")
with col2:
    status_color = "normal" if enable_alerts else "off"
    st.metric(label="Alert Engine", value="ACTIVE" if enable_alerts else "PAUSED")
with col3:
    st.metric(label="Target Asset", value="IDEA.NS")

st.markdown("---")

# ==========================================
# 4. LIVE MONITOR LOGIC
# ==========================================
st.subheader("📡 Live Market Feed")
log_placeholder = st.empty()
last_alert_time = 0

# The "Start" Button
if st.button("🔴 START LIVE MONITOR", type="primary"):

    st.toast("Connecting to Data Feed...", icon="⏳")

    # Initialize "Memory" to prevent spamming
    last_signal_state = "NEUTRAL"

    try:
        # Loop for 100 steps (Simulating 100 minutes)
        for i in range(100):

            # --- A. SIMULATE DATA (Since we are in standalone app) ---
            # In real version, you would call: data = get_hybrid_data()
            price = 100 + np.random.randint(-5, 5) + (i * 0.2)
            rsi = np.random.randint(20, 80)

            # Determine Signal based on RSI
            current_signal = "NEUTRAL"
            if rsi > 70: current_signal = "SELL"
            elif rsi < 30: current_signal = "BUY"

            # --- B. ALERT LOGIC (The Bridge) ---
            alert_sent = False

            if enable_alerts:
                # Check BUY
                if check_buy and current_signal == "BUY" and last_signal_state != "BUY":
                    st.toast(f"BUY SIGNAL DETECTED! ({rsi})", icon="🚀")
                    email_status = send_email_alert("IDEA.NS", "BUY", price)
                    if email_status == True:
                        st.success(f"✅ Email sent for BUY at {price}")
                    else:
                        st.error(f"❌ Email Failed: {email_status}")
                    alert_sent = True

                # Check SELL
                elif check_sell and current_signal == "SELL" and last_signal_state != "SELL":
                    st.toast(f"SELL SIGNAL DETECTED! ({rsi})", icon="🔴")
                    email_status = send_email_alert("IDEA.NS", "SELL", price)
                    if email_status == True:
                        st.success(f"✅ Email sent for SELL at {price}")
                    alert_sent = True

            # --- C. UPDATE UI ---
            log_placeholder.code(
                f"STEP {i} | Price: {price:.2f} | RSI: {rsi} | Signal: {current_signal}"
            )

            # Update memory
            if current_signal != "NEUTRAL":
                last_signal_state = current_signal

            time.sleep(1.5) # Wait 1.5 seconds between updates

    except KeyboardInterrupt:
        st.warning("Monitor Stopped.")

In [ ]:
# 1. Install Streamlit & Localtunnel (Just in case)
!pip install -q streamlit
!npm install -g localtunnel

# 2. Get the Password
import urllib
print("COPY THIS PASSWORD:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# 3. Run the App
print("CLICK THE LINK BELOW AND PASTE THE PASSWORD:")
!streamlit run app.py & npx localtunnel --port 8501